# 1. 인공지능과 가위바위보 하기

## 1. 인공지능과 가위바위보 하기

### 간단한 이미지 분류기
---
이번 시간에는 이미지를 분류하는 간단한 인공지능을 만들어 볼 것이다. 숫자 손글씨를 인식하는 인공지능을 만들 것인데, 숫자는 0~9까지 총 10개의 클래스(class)만 인식을 하면 된다. 이렇게 클래스가 몇 개 안되는 경우, 인공지능은 간단하게 이미지를 분류해 낼 수 있다.